In [25]:
import numpy as np, pandas as pd
import scipy.stats as st

### Príklad(1) (H0)
Na silnicích první třídy se při nehodách umíralo se stejnou pravděpodobností jako dálnicích.
### Alternatívna (H1)
Na silnicích první třídy se při nehodách **ne**umíralo se stejnou pravděpodobností jako na dálnicích.

In [45]:
df = pd.read_pickle("accidents.pkl.gz")
df_orig = df.copy()

df = df[["p36", "p13a"]]

Vytvorenie pomocných stĺpcov true/false

In [48]:
df = df.query("p36 == 1 or p36 == 0")
#cesty prvej triedy
df["road1"] = df['p36'] == 1 
# vybrat dialnicu
df["highway"] = df['p36'] == 0 
# fatalne nehody
df["deadly"] = df["p13a"] > 0

Crosstabulka určí počet smrteľných nehôd na ceste prvej triedy, ostatné sú na diaľnici

In [50]:
crosstab_road1 = pd.crosstab(df['road1'], df['deadly'])
crosstab_road1

deadly,False,True
road1,,
False,24293,166
True,78618,911


porovnanie s tabuľkou očakávaných frekvencií

In [55]:
stat, p_value, dof, expected = st.chi2_contingency(crosstab_road1)
expected

array([[24205.67901104,   253.32098896],
       [78705.32098896,   823.67901104]])

Chi kvadrant test pre hypotézu

In [101]:
prob = 0.95
critical = st.chi2.ppf(prob, dof)
if abs(stat) < critical:
	print("Hypotéza 'H0' nebola zamietnutá platí 'H1'")
else:
	print("Hypotéza 'H0' bola zamietnutá. Platí 'H1'")
print("P hodnota: " + str(p_value))

Hypotéza 'H0' bola zamietnutá. Platí 'H1'
P hodnota: 3.6067450279444316e-10


Platí H1 čo znamená že s pravdepodobnosťou 95% sa na cestách 1. triedy a na dialniciach neumieralo s rovnakou pravdepodobnosťou.

### Príklad(2) (H0)
Při nehodách vozidel značky Škoda je škoda na vozidle nižší než při nehodách vozidel Audi.
### Alternatívna (H1)
Při nehodách vozidel značky Škoda je škoda na vozidle **stejná nebo vyšší** než při nehodách vozidel Audi.

potrebné časti tabuľky

In [59]:
df = df_orig.copy()
df = df[["p45a","p53"]]

mediány a priemery pre vozy

In [87]:
medians = df.groupby('p45a')['p53'].median()
means = df.groupby('p45a')['p53'].mean()
audi_median = medians[2]
skoda_median = medians[39]
audi_average = means[2]
skoda_average = means[39]
print("medián audi: " + str(audi_median))
print("medián škoda: " + str(skoda_median))

medián audi: 300.0
medián škoda: 200.0


stredná hodnota nám naznačuje že obvyklé škody na audi sú vyššie


**Aproximácia Alexandra-Governa** testuje rovnosť k nezávislých priemerov zoči-voči heterogenite rozptylu.

In [92]:
audi = df['p53'].where(df['p45a']==2).dropna().tolist()
skoda = df['p53'].where(df['p45a']==39).dropna().tolist()

p = st.alexandergovern(audi, skoda)
p

AlexanderGovernResult(statistic=545.472551102053, pvalue=1.215910364472692e-120)

P hodnota vyšla výrazne spadá výrazne pod hladinu 0.05 čo znamená že zamietame hypotézu **H0** a platí **H1**

**statistics = 545.472551102053** poukazuje na veľký rozptyl hodnôt